Here we will compare and analyze how major countries did in the 2021 tokyo olympcs. 

In [1]:
import pandas as pd 
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px

import warnings
warnings.filterwarnings('ignore')

In [2]:
teams = pd.read_excel('../input/2021-olympics-in-tokyo/Teams.xlsx')
athletes = pd.read_excel('../input/2021-olympics-in-tokyo/Athletes.xlsx')
coaches = pd.read_excel('../input/2021-olympics-in-tokyo/Coaches.xlsx')
medals = pd.read_excel('../input/2021-olympics-in-tokyo/Medals.xlsx')
entriesgenders = pd.read_excel('../input/2021-olympics-in-tokyo/EntriesGender.xlsx')
continents = pd.read_excel('../input/continent-list-for-2021-olympics-in-tokyo-dataset/Continent List.xlsx')

# Analysis
First, lets see a gender ratio.

In [3]:
entriesgenders.head()

,Discipline,Female,Male,Total
0,3x3 Basketball,32,32,64
1,Archery,64,64,128
2,Artistic Gymnastics,98,98,196
3,Artistic Swimming,105,0,105
4,Athletics,969,1072,2041


In [4]:
male = entriesgenders["Male"].sum()
female = entriesgenders["Female"].sum()

In [5]:
fig = go.Figure(data =[go.Pie(labels=['Male','Female'], values=[male,female])],)
fig.update_traces(hoverinfo='value', textinfo='percent')
fig.update_layout(template="seaborn", font_family="Rockwell", font_size=16, title="Male/Female athletes",
                                                                                 title_font_size=24, hoverlabel=dict(bgcolor="white", font_size=12, font_family="Rockwell"))
fig.show()

This results make sense because almost every event at the olympics include both male and females. Where are the most athletes from?

In [6]:
athletes.head()

,Name,NOC,Discipline
0,AALERUD Katrine,Norway,Cycling Road
1,ABAD Nestor,Spain,Artistic Gymnastics
2,ABAGNALE Giovanni,Italy,Rowing
3,ABALDE Alberto,Spain,Basketball
4,ABALDE Tamara,Spain,Basketball


In [7]:
athetes = athletes.replace("People's Republic of Chine", "China")
athletes = athletes.replace("United States of America", "USA")
athletes_count = athletes["NOC"].value_counts()

In [8]:
athletes_count.head()

USA                           615
Japan                         586
Australia                     470
People's Republic of China    401
Germany                       400
Name: NOC, dtype: int64

In [9]:
fig = go.Figure(go.Bar(x=athletes_count.values[:10],y=athletes_count.index[:10],orientation='h'))

fig.update_layout(template="seaborn", font_family="Rockwell", font_size=16, title="Nations with most athletes",
                title_font_size=24, hoverlabel=dict(bgcolor="white",font_size=12,font_family="Rockwell"))

fig.update_traces(textfont_size=12, hovertemplate='<b>%{y} - %{x} athletes</b><extra></extra>')

fig.show()

Of course from the largest countries... And how about continents?

In [10]:
continents.head()

,Continent,Country
0,Asia,Afghanistan
1,Europe,Albania
2,Africa,Algeria
3,Oceania,American Samoa
4,Europe,Andorra


In [11]:
continents = continents.rename(columns = {"Country":"NOC"})

In [12]:
continents.head()

,Continent,NOC
0,Asia,Afghanistan
1,Europe,Albania
2,Africa,Algeria
3,Oceania,American Samoa
4,Europe,Andorra


In [13]:
continents = continents.replace("United States", "USA")
continents = continents.replace("Russian Olympic Committee", "ROC")
continents = continents.replace("Russian Olympic Committee", "ROC")
continents = continents.fillna('Other')
continent_athletes = pd.merge(continents, athletes, on = "NOC")

In [14]:
continent_athletes.head()

,Continent,NOC,Name,Discipline
0,Asia,Afghanistan,ANWARI Fahim,Swimming
1,Asia,Afghanistan,MANSOURI Farzad,Taekwondo
2,Asia,Afghanistan,NOOR ZAHI Sha Mahmood,Athletics
3,Asia,Afghanistan,YOUSOFI Kimia,Athletics
4,Asia,Afghanistan,YOVARI Mahdi,Shooting


In [15]:
continent_athletes = continent_athletes["Continent"].value_counts()

In [16]:
fig = go.Figure(go.Treemap(values = continent_athletes.values,labels = continent_athletes.index,
                parents = ['','','','','','','','',''],textinfo = "label+percent entry"))

fig.update_layout(template="seaborn", font_family="Rockwell", font_size=16, title="Athletes distribution",
                title_font_size=24, margin = dict(t=50, l=25, r=25, b=25),
                hoverlabel=dict(bgcolor="white",font_size=12,font_family="Rockwell"))

fig.update_traces(hoverinfo='value')

fig.show()

Europe make up almost half of the athletes. What are the main disciplines?

In [17]:
entriesgenders = entriesgenders.sort_values("Total", ascending=True)

In [18]:
fig = go.Figure(go.Bar(x=entriesgenders["Total"],y=entriesgenders["Discipline"],orientation='h'))
                
fig.update_layout(template="seaborn", font_family="Rockwell", font_size=8, height=1000,
                title="Athletes per disciplines",xaxis_title="Athletes", title_font_size=24,
                hoverlabel=dict(bgcolor="white",font_size=12,font_family="Rockwell"))

fig.update_traces(textfont_size=12, hovertemplate='<b>%{y} - %{x} athletes</b><extra></extra>')

fig.show()

Athletics are on the top. Now lets see something more interesting. How did the nations perform based on medal count?

In [19]:
medals.head()

,Rank,Team/NOC,Gold,Silver,Bronze,Total,Rank by Total
0,1,United States of America,39,41,33,113,1
1,2,People's Republic of China,38,32,18,88,2
2,3,Japan,27,14,17,58,5
3,4,Great Britain,22,21,22,65,4
4,5,ROC,20,28,23,71,3


In [20]:
medals = medals.replace("People's Republic of China", "China")
medals = medals.replace("United States of America", "USA")
athletes_count = athletes_count.to_frame()
athletes_count.index.name = "Team/NOC"

In [21]:
athletes_count.head()

,NOC
Team/NOC,
USA,615
Japan,586
Australia,470
People's Republic of China,401
Germany,400


In [22]:
total_medal = pd.merge(medals, athletes_count, on = "Team/NOC")

In [23]:
total_medal.head()

,Rank,Team/NOC,Gold,Silver,Bronze,Total,Rank by Total,NOC
0,1,USA,39,41,33,113,1,615
1,3,Japan,27,14,17,58,5,586
2,4,Great Britain,22,21,22,65,4,366
3,5,ROC,20,28,23,71,3,318
4,6,Australia,17,7,22,46,6,470


In [24]:
total_medal = total_medal.rename(columns={"NOC":"Athletes Count"})
top10medal = total_medal[:10].sort_values(by = "Total", ascending=False)

In [25]:
fig = go.Figure(data=[
    go.Bar(x=top10medal["Team/NOC"], y=top10medal["Bronze"], name="Bronze" ,hovertext=['Bronze']),
    go.Bar(x=top10medal["Team/NOC"], y=top10medal["Silver"], name="Silver" ,hovertext=['Silver']),
    go.Bar(x=top10medal["Team/NOC"], y=top10medal["Gold"], name="Gold" ,hovertext=['Gold'])])

fig.update_layout(template="seaborn", colorway=["#ac896a","#c1c1c1","#e6b11a"], font_family="Rockwell", 
                font_size=16, title="TOP 10",title_font_size=24, barmode="stack",
                hoverlabel=dict(bgcolor="white",font_size=12,font_family="Rockwell"))

fig.update_traces(textfont_size=12, hoverinfo='y')

fig.show()